# Dataload

In [1]:
!pip install roboflow
!pip install pytorch
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 117.0 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and 

In [ ]:

from roboflow import Roboflow
import shutil


rf = Roboflow(api_key="APIKEY_HERE")
project = rf.workspace("ball-tracking-vcxpr").project("court-detection-bxo2j-zts3d")
version = project.version(7)
dataset = version.download("coco")



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Court-Detection-7 in coco:: 100%|██████████| 761/761 [00:00<00:00, 2433.23it/s]


# Torch Setup

In [3]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
import cv2
import os
import numpy as np
import json

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


# Torch Dataset

In [5]:
class KeyPointsDataset(Dataset):
    def __init__(self, data_dir, data_file):
        self.data_dir = data_dir
        with open(data_file, 'r') as f:
            self.data = json.load(f)

        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
        ])


    def __len__(self):
        return len(self.data["images"])

    def __getitem__(self, idx):
        img_path = self.data["images"][idx]["file_name"]
        img = cv2.imread(os.path.join(self.data_dir, img_path))
        h, w = img.shape[:2]
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)

        kps = np.array(self.data["annotations"][idx]["keypoints"])
        kps = kps.astype(np.float32)
        kps[0::3] *= (224 / w)
        kps[1::3] *= (224 / h)

        kps = torch.tensor(kps, dtype=torch.float32)

        return img, kps







In [6]:
train_data_dir = "Court-Detection-7/train"
val_data_dir = "Court-Detection-7/valid"
file_path = "_annotations.coco.json"
train_dataset = KeyPointsDataset(train_data_dir, os.path.join(train_data_dir, file_path))
val_dataset = KeyPointsDataset(val_data_dir, os.path.join(val_data_dir, file_path))

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)

# Create Model

In [7]:
model = models.resnet50(pretrained=True)

keypoint_counds = 12
model.fc = torch.nn.Linear(model.fc.in_features, keypoint_counds * 3)


The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 177MB/s]


In [8]:
model = model.to(device)

# Train Model

In [9]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
epoches = 125
for epoch in range(epoches):
    for i, (images, keypoints) in enumerate(train_loader):
        images = images.to(device)
        keypoints = keypoints.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, keypoints)
        loss.backward()
        optimizer.step()

        if (i + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epoches}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

Epoch [1/125], Step [10/34], Loss: 8519.5195
Epoch [1/125], Step [20/34], Loss: 6261.2485
Epoch [1/125], Step [30/34], Loss: 4019.6311
Epoch [2/125], Step [10/34], Loss: 2248.8945
Epoch [2/125], Step [20/34], Loss: 1425.1730
Epoch [2/125], Step [30/34], Loss: 916.3698
Epoch [3/125], Step [10/34], Loss: 877.1524
Epoch [3/125], Step [20/34], Loss: 418.3758
Epoch [3/125], Step [30/34], Loss: 500.0815
Epoch [4/125], Step [10/34], Loss: 376.3976
Epoch [4/125], Step [20/34], Loss: 259.8979
Epoch [4/125], Step [30/34], Loss: 135.2102
Epoch [5/125], Step [10/34], Loss: 75.2014
Epoch [5/125], Step [20/34], Loss: 276.9234
Epoch [5/125], Step [30/34], Loss: 269.8593
Epoch [6/125], Step [10/34], Loss: 268.2897
Epoch [6/125], Step [20/34], Loss: 269.5298
Epoch [6/125], Step [30/34], Loss: 340.0228
Epoch [7/125], Step [10/34], Loss: 61.1354
Epoch [7/125], Step [20/34], Loss: 63.9493
Epoch [7/125], Step [30/34], Loss: 195.7274
Epoch [8/125], Step [10/34], Loss: 228.3146
Epoch [8/125], Step [20/34], L

In [11]:
torch.save(model.state_dict(), 'keypoints_model_v4.pth')

In [12]:
model.eval()  # Set the model to evaluation mode
total_mae = 0.0
total_mse = 0.0
num_samples = 0

with torch.no_grad():
    for inputs, targets in val_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Make predictions
        predictions = model(inputs)

        # Calculate batch metrics
        batch_mae = torch.mean(torch.abs(predictions - targets))
        batch_mse = torch.mean((predictions - targets) ** 2)

        # Accumulate totals
        total_mae += batch_mae.item() * inputs.size(0)
        total_mse += batch_mse.item() * inputs.size(0)
        num_samples += inputs.size(0)

# Calculate overall metrics
avg_mae = total_mae / num_samples
avg_mse = total_mse / num_samples
avg_rmse = torch.sqrt(torch.tensor(avg_mse)).item()

print(f'Average MAE: {avg_mae:.4f}')
print(f'Average MSE: {avg_mse:.4f}')
print(f'Average RMSE: {avg_rmse:.4f}')

Average MAE: 1.6087
Average MSE: 61.9435
Average RMSE: 7.8704
